In [28]:
import pandas as pd
import numpy as np


reviews_path = 'yelp_data/yelp_academic_dataset_review.json'
business_path = 'yelp_data/yelp_academic_dataset_business.json'

In [29]:
reviews_json = pd.read_json(reviews_path, lines=True, chunksize=1000000)
business_df = pd.read_json(business_path, lines=True)

In [30]:
"""
Gets the indexes of all the relevant businesses that are restaurants/bars/coffee shops
"""

key_words = ["Restaurants", "Food", "Coffee", "Tea", "Bars"]
selected_business = []
for ind, val in enumerate(business_df.categories.values):
    if val == None:
        continue
    if any(x in val for x in key_words):
        selected_business.append(ind)

In [31]:
business_subset = business_df.iloc[selected_business]
selected_ids = business_subset.business_id.values

In [32]:
#Gets first 1,000,000 reviews as reviews_json is a generator
rev_df = next(reviews_json)

In [33]:
"""
Converts the reviews dataframe to contain only the selected businesses
"""
reviews = rev_df[["text", "business_id"]]
reviews = reviews[reviews['business_id'].isin(selected_ids)]

In [36]:
"""
Filters out very long reviews; this speeds up gpt2 fine-tuning, will have generate lengths of about 100
"""
res = []
for i in reviews.text.values:
    if len(i) < 150:
        res.append(i)

In [37]:
len(res)

86085

In [38]:
"""
Saves a txt file with plain text of relevant reviews for use in finetuning GPT2 model
"""
np.savetxt('reviews.txt', res, fmt='%s')